<a href="https://colab.research.google.com/github/andrew128/graph-rag-playground/blob/main/graph_rag_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install networkx transformers torch faiss-cpu numpy ollama

import networkx as nx
from transformers import AutoModel, AutoTokenizer
import faiss
import numpy as np
from ollama import chat
from ollama import ChatResponse
from typing import List

In [15]:
class BasicGraphRag():
  """
  This graph rag will take in input text, chunk it, embed it, and add to a graph

  TODO: include logging and metrics
  TODO: think about how to better refactor this with an interface for GraphRagApp
    or maybe an abstract class for including logging and generating report metric implementations
    - yeah like a method generate() that calls generate_impl() and generate() has the latency logging
    - generate_impl() is overriden by the base classes
    - would probably need to move the class variables into a GraphRagConfig class
    - maybe have some top level interface functions and that's it for the graph rag abstract class
    - then have some other evaluator class that takes in the data, have some metrics, runs the graph rag app, generates report
    - but dont create this abstract class until i start doing more than one app. b/c for example llm idk what to do. want llm for different things.
    - or different llms for different parts
  """
  def __init__(self):
    self.graph = nx.Graph()
    self.embedding = None
    self.vector_store = None
    self.llm =
    self.message_history = []

  def query(self, text: str) -> str:
    """
    Top level user function to query for
    """
    relevant_texts = self._retrieve(text)
    return self._generate(text, relevant_texts)

  def _generate(self, text: str, retrieved_texts: List[str]) -> str:
    # use ollama chat to prompt and generate a response
    pass

  def index(self, text: str) -> None:
    # chunk, then embed each chunk, then store in vector db
    pass

  def _retrieve(self, text: str) -> List[str]:
    input_embedding = self._embed(text)
    # retrieve from vector store
    pass

  def _embed(self, text: str) -> List[int]: # maybe turn this into a numpy array
    # call embedding api
    pass

In [18]:
graph_rag_app = BasicGraphRag()
graph_rag_app.index('bob is a blob')
graph_rag_app.query('what is bob')


In [ ]:
class Evaluator():
  """
  Class for running evaluation metrics
  """
  def __init__(self):
    pass

  def evaluate(self, text, graph_rag):
    pass